# Predict Tomorrow's Closing Price of a Stock
This notebook was built to satisfy a homework assignment for the Deep Learning class at the University of Colorado, Boulder. Below I will attempt to predict the following day's close for a stock. Stock trading is a highly valuable exercise, so if we can develop a model would be priceless. I will use LSTM and GRU recurrent models to make two attempts at getting a working model. I will also go through several optimizers to see which performs best on each of the classes of units (LSTM and GRU).

#### The data:
Stock market data is both well understood, and readily available. Each stock has a closing price for each day of trading, the data is regimented, and the SEC watches over the system for irregularities. We will get our data from the Yahoo Finance module so that we have a reputable source. There isn't much exploratory data analysis to do, but we will characterize and inspect our data before modeling.

Citation:
* Aroussi, R. Yfinance 0.2.36 Documentation. PyPI YFinance. https://pypi.org/project/yfinance/ 


#### Work to do:
1. Load Libraries.
2. Build Function to get stock data.
3. EDA on a few random stocks.
4. Build the LSTM Model
5. Train and Test Model on 3 Optimizers
6. Build the GRU Model
7. Train and Test Model on 3 Optimizers
8. Collate Results and Visualize
9. Conclusion:

#### CAVEAT: 
Because yfinance scrapes data for a few of it’s functions, you sometimes run the risk of getting rate limited or blacklisted for too many scraping attempts. This is a risk that’s always present when trying to scrape websites, but when you’re building applications trading real money on top of infrastructure that might be making a lot of data requests, the risk/reward calculus changes. *Do not use this script to trade real money*.

# Load Libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
!pip install yfinance
import yfinance  as yf
from sklearn.preprocessing import RobustScaler
from collections import deque
 
import tensorflow as tf
print("TF Version:", tf.__version__)
from keras.models import Sequential
from keras.layers import Dense, GRU, LSTM, Dropout, Input
import os, warnings, matplotlib.pyplot as plt
warnings.filterwarnings('ignore')



#### CAVEAT: 
Because yfinance scrapes data for a few of it’s functions, you sometimes run the risk of getting rate limited or blacklisted for too many scraping attempts. This is a risk that’s always present when trying to scrape websites, but when you’re building applications trading real money on-top of infrastructure that might be making a lot of data requests, the risk:reward changes. Do not use this script to trade real money.



# Build Function to Gather Stock Data

In [ ]:
# Function to extract stock data from yfinance
def Extract_Data(Stock = 'BA',   intervalmos = 12, interval = '1d'):
    STOCK = Stock
    now = datetime.today() # Current date  
    DATE_NOW = now.strftime('%Y-%m-%d') # Extract today's date as a string 
    #set the interval start date string
    if int(intervalmos) > 0:
        DATE_THEN = now - relativedelta(months=intervalmos)
        DATE_THEN = DATE_THEN.strftime('%Y-%m-%d')
    else:
        DATE_THEN = now - relativedelta(months=12)
        DATE_THEN = DATE_THEN.strftime('%Y-%m-%d')
    #load the data into a dataframe and return it
    raw_price_df = yf.download(STOCK, start=DATE_THEN, end=DATE_NOW, interval=interval)
    return raw_price_df

# test the function with Boeing for 12 mos, at 1hr interval
# available intervals: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo
# important to note that the 1m data is only retrievable for the last 7 days, and anything intraday (interval <1d) only for the last 60 days
testdata = Extract_Data('BA',12,'1d')
testdata

Great. We have stock data for a stock, for each trading day in the last year (251 days = 50 weeks * 5 days +1). 

# Cursory Exploratory Data Analysis

Let's see if there are any NaNs, empty cells, and look to see if the data is normally distributed for all price metrics. The volume metric should have a gamma or exponential distribution.

In [ ]:
# Test for empty cells or NaN
print("Any NaNs?")
print(testdata.isna().sum())
print('\nAny Empty Cells?')
print(testdata.info())

In [ ]:
fig,ax = plt.subplots(2,3,figsize=(12,6))
j=0
h=0
for i in range(6):
    ax[j,h].hist(testdata.iloc[:,i])
    ax[j,h].set_xlabel(testdata.columns[i])
  
    if j == 1:
        h += 1
        j = 0
    else:
        j +=1
plt.show()

As expected, the data is clean and regimented. All the price metrics have a normal distribution. The volume metric has a gamma or exponential distribution. We can change the testdata function to test any stock we like. Moving on...

Before we build the model, let's build a better visualizer and a function to preprocess data for the ensuing model.

In [ ]:
# function to visualize the extracted closing price data
def Visualize_Data(df, Stock,  figsize = (20,9)):
    #plt.style.use(style='ggplot')
    plt.figure(figsize=figsize)
    plt.plot(df['Close'],color='b')
    plt.xlabel('Date')
    plt.ylabel('Price in $')
    plt.legend([f'Price of {Stock} Share'])
    plt.title(f'Share Prices for {Stock} Stock')
    plt.show()

#test the visualizer
Visualize_Data(testdata, 'BA')


We need another function to prepare the data for the models below so we only write this once. We'll also make the standard scaler a global function to se can use it inside the function and outside later.

In [ ]:
#set scaler as a global variable to access inside of the function and outside later...
global scaler
scaler = RobustScaler()

# Function to process data set for training
def Prepare_Data(dataframe, days, stepper):
    # copy the dataframe before we play with it
    df = dataframe.copy()
    
    # Cleave off the number of days to predict from bottom of dataframe
    df = df.iloc[:(df.shape[0]-stepper+1),:]
    # print(df.shape)
    
    # Pre-process the DataFrame
    df = df.drop(['Open', 'High', 'Low', 'Adj Close',  'Volume'], axis=1)
    df['date'] = df.index
    
    # use global scaler function to fit between 0 and 1
    df['scaled_close'] = scaler.fit_transform(np.expand_dims(df['Close'].values, axis=1))
    #print(df.scaled_close)
    
    # Engineer the DataFrame to add future and start defining the last sequence
    df['future'] = df['scaled_close'].shift(-(days))
    #print(len(df.future))
    last_sequence = np.array(df[['scaled_close']].tail(days))
    #print(last_sequence)
    
    # Just in case, drop and NaNs
    df.dropna(inplace=True)  
    
    # Define Variables for sequencing
    sequence_data = []
    sequences = deque(maxlen=days)
    #print(len(df.future))
    
    # Generate arrays of close data and dates up to the last sequence...
    for entry, target in zip(df[['scaled_close','date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == days:
            sequence_data.append([np.array(sequences), target])
    #print(sequence_data)
    
    # Modify last sequence to remove close info and flip to array
    last_sequence = list([x[:1] for x in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # build X and Y training set
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

    # convert X and Y to numpy arrays for compatibility
    X = np.array(X)
    Y = np.array(Y)

    return last_sequence, X, Y

# Test the preprocessor and view shapes to ensure they add up...
a, b = 12,1
te = 252 - a -  1 + b
print("Test days:",te)
tested = Prepare_Data(testdata,a,b)
print("last_sequence",tested[0].shape,"\n X:",tested[1].shape,"\n Y:",tested[2].shape, "\nTotal Days Match:", tested[0].shape[0]+tested[1].shape[0]-a == te )
#tested[1]

Great. With data at the ready, we can move on to building the Models, Training, etc.

# Build LSTM Model

To begin, we'll define a function to build a simple Sequential model with 2 layers of LSTM and Dropout that returns a single value. We'll build the function with loss, optimizer, epoch, and batch size inputs to iterate through several options and record results easily.

In [ ]:

def Train_Model(x_train, y_train, NUMBER_of_STEPS_BACK, BATCH_SIZE, UNITS, EPOCHS, DROPOUT, OPTIMIZER, LOSS):

    model = Sequential([
        Input(shape=(NUMBER_of_STEPS_BACK,1),batch_size= BATCH_SIZE),
        LSTM(UNITS, return_sequences=True), # use_bias=True,stateful=True, recurrent_initializer='orthogonal'),
        Dropout(DROPOUT),
        LSTM(UNITS, return_sequences=False),
        Dropout(DROPOUT),
        Dense(1)
        ])
    #model.summary()
    model.compile(loss=LOSS, optimizer=OPTIMIZER)
    model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)


    return model

In [ ]:
# Define the Stock to model, and YFinance pull characteristics
STOCK = 'BA' # Specify the ticker symbol 
LENGTH = 12 # Length of the extracted data in months
INTERVAL = '1d' # Interval (see options above)
PREDICTION_STEPS = 5 # Number of int(days) the model(s) will predict. To predict the next three days change to 3.

#Set Variable Parameters/Hyperparameters
if INTERVAL == '4h':
    NUMBER_of_STEPS_BACK = 30 * 2 # Number of days * 3 times per day that the model will be trained 
elif INTERVAL == '1h':
    NUMBER_of_STEPS_BACK = 30 * 2 # Number of days * 7 hrs per day that the model will be trained 
else: # INTERVAL == '1d'
    NUMBER_of_STEPS_BACK = 17  # Number of days back that the model will be trained   

# Visualize Data
raw_price_df = Extract_Data(Stock = STOCK, intervalmos = LENGTH,interval =    INTERVAL )
Visualize_Data(raw_price_df, Stock = STOCK, )


Now that we have stock data to model, a modeling function, and can see it, we define parameters for the first model and see what we get.

In [ ]:
#Set Constant Parameters/Hyperparameters
BATCH_SIZE = 25 # Number of training samples that will be passed per step
DROPOUT = 0.25 # To improve performance by regularization  
UNITS = 60 # Number of LSTM neurons in each layer
EPOCHS = 10 # Number of iterations for model training
LOSS='mean_squared_error' # Loss Methodology 
OPTIMIZER='adam' # Optimizer 

# Make Prediction
predictions = []

for step in range(PREDICTION_STEPS):
    stepped = PREDICTION_STEPS - step
    last_sequence, x_train, y_train = Prepare_Data(raw_price_df, NUMBER_of_STEPS_BACK, stepped)
    #print("Sequence Pull Shape:",last_sequence.shape)
    x_train = x_train[:, :, :1].astype(np.float32)

    model = Train_Model(x_train, y_train, NUMBER_of_STEPS_BACK, BATCH_SIZE, UNITS, EPOCHS, DROPOUT, OPTIMIZER, LOSS)

    last_sequence = last_sequence[-(NUMBER_of_STEPS_BACK):]
    last_sequence = np.expand_dims(last_sequence, axis=0)
    prediction = model.predict(last_sequence)
    print(prediction)
    predicted_price = scaler.inverse_transform(prediction)[0][0]
  
    predictions.append(round(float(predicted_price), 2))
    
# Print Predictions
if len(predictions) > 0:
    predictions_list = ['$'+str(d) for d in predictions]
    predictions_str = ', '.join(predictions_list)
    message = f'{STOCK} share price prediction(s) for next {len(predictions)} day(s) {predictions_str}'
    print(message)

LSTMdf = pd.DataFrame({"Date":list(raw_price_df.index)[-PREDICTION_STEPS:],
                       "Actual Close":[round(x,2) for x in list(raw_price_df.Close.tail(PREDICTION_STEPS).values)],
                       "Predicted Close":predictions 
                       })
LSTMdf['Difference'] = [round(x,2) for x in np.array(LSTMdf['Actual Close']).astype(float)-predictions]
LSTMdf["Model"] = 'LSTM'
LSTMdf["Optimizer"] = "Adam"
LSTMdf["Batch Size"] = BATCH_SIZE

LSTMdf

In [ ]:

#Set Constant Parameters/Hyperparameters
BATCH_SIZE = 15 # Number of training samples that will be passed per step
DROPOUT = 0.25 # To improve performance by regularization  
UNITS = 60 # Number of LSTM neurons in each layer
EPOCHS = 10 # Number of iterations for model training
LOSS='mean_squared_error' # Loss Methodology 
OPTIMIZER='RMSprop' # Optimizer 

# Make Prediction
predictions = []

for step in range(PREDICTION_STEPS):
    stepped = PREDICTION_STEPS - step
    last_sequence, x_train, y_train = Prepare_Data(raw_price_df, NUMBER_of_STEPS_BACK, stepped)
    #print("Sequence Pull Shape:",last_sequence.shape)
    x_train = x_train[:, :, :1].astype(np.float32)

    model = Train_Model(x_train, y_train, NUMBER_of_STEPS_BACK, BATCH_SIZE, UNITS, EPOCHS, DROPOUT, OPTIMIZER, LOSS)

    last_sequence = last_sequence[-(NUMBER_of_STEPS_BACK):]
    last_sequence = np.expand_dims(last_sequence, axis=0)
    prediction = model.predict(last_sequence)
    print(prediction)
    predicted_price = scaler.inverse_transform(prediction)[0][0]
  
    predictions.append(round(float(predicted_price), 2))
    
# Print Predictions
if len(predictions) > 0:
    predictions_list = ['$'+str(d) for d in predictions]
    predictions_str = ', '.join(predictions_list)
    message = f'{STOCK} share price prediction(s) for next {len(predictions)} day(s) {predictions_str}'
    print(message)

LSTM2df = pd.DataFrame({"Date":list(raw_price_df.index)[-PREDICTION_STEPS:],
                       "Actual Close":[round(x,2) for x in list(raw_price_df.Close.tail(PREDICTION_STEPS).values)],
                       "Predicted Close":predictions
                       })
LSTM2df['Difference'] = [round(x,2) for x in np.array(LSTMdf['Actual Close']).astype(float)-predictions]
LSTM2df["Model"] = 'LSTM'
LSTM2df["Optimizer"] = OPTIMIZER
LSTM2df["Batch Size"] = BATCH_SIZE

LSTM2df

In [ ]:
#Set Constant Parameters/Hyperparameters
BATCH_SIZE = 12 # Number of training samples that will be passed per step
DROPOUT = 0.25 # To improve performance by regularization  
UNITS = 60 # Number of LSTM neurons in each layer
EPOCHS = 10 # Number of iterations for model training
LOSS='mean_squared_error' # Loss Methodology 
OPTIMIZER='SGD' # Optimizer 

# Make Prediction
predictions = []

for step in range(PREDICTION_STEPS):
    stepped = PREDICTION_STEPS - step
    last_sequence, x_train, y_train = Prepare_Data(raw_price_df, NUMBER_of_STEPS_BACK, stepped)
    #print("Sequence Pull Shape:",last_sequence.shape)
    x_train = x_train[:, :, :1].astype(np.float32)

    model = Train_Model(x_train, y_train, NUMBER_of_STEPS_BACK, BATCH_SIZE, UNITS, EPOCHS, DROPOUT, OPTIMIZER, LOSS)

    last_sequence = last_sequence[-(NUMBER_of_STEPS_BACK):]
    last_sequence = np.expand_dims(last_sequence, axis=0)
    prediction = model.predict(last_sequence)
    print(prediction)
    predicted_price = scaler.inverse_transform(prediction)[0][0]
  
    predictions.append(round(float(predicted_price), 2))
    
# Print Predictions
if len(predictions) > 0:
    predictions_list = ['$'+str(d) for d in predictions]
    predictions_str = ', '.join(predictions_list)
    message = f'{STOCK} share price prediction(s) for next {len(predictions)} day(s) {predictions_str}'
    print(message)

LSTM3df = pd.DataFrame({"Date":list(raw_price_df.index)[-PREDICTION_STEPS:],
                       "Actual Close":[round(x,2) for x in list(raw_price_df.Close.tail(PREDICTION_STEPS).values)],
                       "Predicted Close":predictions
                       })
LSTM3df['Difference'] = [round(x,2) for x in np.array(LSTMdf['Actual Close']).astype(float)-predictions]
LSTM3df["Model"] = 'LSTM'
LSTM3df["Optimizer"] = OPTIMIZER
LSTM3df["Batch Size"] = BATCH_SIZE

LSTM3df

Great, now that we have 2 models worth of results let's see how they compare. We'll compute average difference between the predicted close and the actual close and concatenate the two dataframes to get a clear view into the data.

In [ ]:
LSTMav = round(np.mean(LSTMdf.Difference),3)
LSTM2av = round(np.mean(LSTM2df.Difference),3)
LSTM3av = round(np.mean(LSTM3df.Difference),3)
print(f"Averages: LSTM with Adam Optimizer: {LSTMav}  -  LSTM with RMSprop Optimizer: {LSTM2av}  - LSTM with SGD Optimizer: {LSTM3av}")
LSTout = pd.concat([LSTMdf,LSTM2df,LSTM3df]) 
LSTout

The RMSprop (Root Mean Square Propogation) optimizer beat the Adam optimizer and SGD (Stochastic Gradient Descent) out of the box (we didn't specifiy a number of parameters including learning rate for any of them). We should notice that we gave the RMSprop a worse chance by giving it fewer observations in each of the batches of data to use in modeling. Were the difference wholly attributable to batch size, SGD would have won as it has the smallest batch size. We can run a bunch more tests here to improve these models, but we'll move on because this is a showcase of LSTM capabilities not a competition.

# Build GRU Model

The same process will be repeated for GRU units in the model instead of LSTM. Again, we'll build a function to build, compile, and train the models from a series of variable inputs. This will help us iterate through several options with a minimum amount of code.

In [ ]:
def Train_ModelG(x_train, y_train, NUMBER_of_STEPS_BACK, BATCH_SIZE, UNITS, EPOCHS, DROPOUT, OPTIMIZER, LOSS):

    model = Sequential([
        Input(shape=(NUMBER_of_STEPS_BACK,1),batch_size= BATCH_SIZE),
        GRU(UNITS, return_sequences=True), # use_bias=True,stateful=True, recurrent_initializer='orthogonal'),
        Dropout(DROPOUT),
        GRU(UNITS, return_sequences=False),
        Dropout(DROPOUT),
        Dense(1)
        ])
    #model.summary()
    model.compile(loss=LOSS, optimizer=OPTIMIZER)
    model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)


    return model

In [ ]:
#Set Constant Parameters/Hyperparameters
BATCH_SIZE = 25 # Number of training samples that will be passed per step
DROPOUT = 0.25 # To improve performance by regularization  
UNITS = 60 # Number of GRU neurons in each layer
EPOCHS = 10 # Number of iterations for model training
LOSS='mean_squared_error' # Loss Methodology 
OPTIMIZER='adam' # Optimizer 

# Make Prediction
predictions = []

for step in range(PREDICTION_STEPS):
    stepped = PREDICTION_STEPS - step
    last_sequence, x_train, y_train = Prepare_Data(raw_price_df, NUMBER_of_STEPS_BACK, stepped)
    #print("Sequence Pull Shape:",last_sequence.shape)
    x_train = x_train[:, :, :1].astype(np.float32)

    modelg = Train_ModelG(x_train, y_train, NUMBER_of_STEPS_BACK, BATCH_SIZE, UNITS, EPOCHS, DROPOUT, OPTIMIZER, LOSS)

    last_sequence = last_sequence[-(NUMBER_of_STEPS_BACK):]
    last_sequence = np.expand_dims(last_sequence, axis=0)
    prediction = modelg.predict(last_sequence)
    print(prediction)
    predicted_price = scaler.inverse_transform(prediction)[0][0]
  
    predictions.append(round(float(predicted_price), 2))
    
# Print Predictions
if len(predictions) > 0:
    predictions_list = ['$'+str(d) for d in predictions]
    predictions_str = ', '.join(predictions_list)
    message = f'{STOCK} share price prediction(s) for next {len(predictions)} day(s) {predictions_str}'
    print(message)

GRUdf = pd.DataFrame({"Date":list(raw_price_df.index)[-PREDICTION_STEPS:],
                       "Actual Close":[round(x,2) for x in list(raw_price_df.Close.tail(PREDICTION_STEPS).values)],
                       "Predicted Close":predictions
                       })
GRUdf['Difference'] = [round(x,2) for x in np.array(LSTMdf['Actual Close']).astype(float)-predictions]
GRUdf["Model"] = 'GRU'
GRUdf["Optimizer"] = OPTIMIZER
GRUdf["Batch Size"] = BATCH_SIZE

GRUdf

In [ ]:
#Set Constant Parameters/Hyperparameters
BATCH_SIZE = 15 # Number of training samples that will be passed per step
DROPOUT = 0.25 # To improve performance by regularization  
UNITS = 60 # Number of LSTM neurons in each layer
EPOCHS = 10 # Number of iterations for model training
LOSS='mean_squared_error' # Loss Methodology 
OPTIMIZER='RMSprop' # Optimizer 

# Make Prediction
predictions = []

for step in range(PREDICTION_STEPS):
    stepped = PREDICTION_STEPS - step
    last_sequence, x_train, y_train = Prepare_Data(raw_price_df, NUMBER_of_STEPS_BACK, stepped)
    #print("Sequence Pull Shape:",last_sequence.shape)
    x_train = x_train[:, :, :1].astype(np.float32)

    modelg = Train_ModelG(x_train, y_train, NUMBER_of_STEPS_BACK, BATCH_SIZE, UNITS, EPOCHS, DROPOUT, OPTIMIZER, LOSS)

    last_sequence = last_sequence[-(NUMBER_of_STEPS_BACK):]
    last_sequence = np.expand_dims(last_sequence, axis=0)
    prediction = modelg.predict(last_sequence)
    print(prediction)
    predicted_price = scaler.inverse_transform(prediction)[0][0]
  
    predictions.append(round(float(predicted_price), 2))
    
# Print Predictions
if len(predictions) > 0:
    predictions_list = ['$'+str(d) for d in predictions]
    predictions_str = ', '.join(predictions_list)
    message = f'{STOCK} share price prediction(s) for next {len(predictions)} day(s) {predictions_str}'
    print(message)

GRU2df = pd.DataFrame({"Date":list(raw_price_df.index)[-PREDICTION_STEPS:],
                       "Actual Close":[round(x,2) for x in list(raw_price_df.Close.tail(PREDICTION_STEPS).values)],
                       "Predicted Close":predictions
                       })
GRU2df['Difference'] = [round(x,2) for x in np.array(LSTMdf['Actual Close']).astype(float)-predictions]
GRU2df["Model"] = 'GRU'
GRU2df["Optimizer"] = OPTIMIZER
GRU2df["Batch Size"] = BATCH_SIZE

GRU2df

In [ ]:
#Set Constant Parameters/Hyperparameters
BATCH_SIZE = 12 # Number of training samples that will be passed per step
DROPOUT = 0.25 # To improve performance by regularization  
UNITS = 60 # Number of LSTM neurons in each layer
EPOCHS = 10 # Number of iterations for model training
LOSS='mean_squared_error' # Loss Methodology 
OPTIMIZER='SGD' # Optimizer 

# Make Prediction
predictions = []

for step in range(PREDICTION_STEPS):
    stepped = PREDICTION_STEPS - step
    last_sequence, x_train, y_train = Prepare_Data(raw_price_df, NUMBER_of_STEPS_BACK, stepped)
    #print("Sequence Pull Shape:",last_sequence.shape)
    x_train = x_train[:, :, :1].astype(np.float32)

    modelg = Train_ModelG(x_train, y_train, NUMBER_of_STEPS_BACK, BATCH_SIZE, UNITS, EPOCHS, DROPOUT, OPTIMIZER, LOSS)

    last_sequence = last_sequence[-(NUMBER_of_STEPS_BACK):]
    last_sequence = np.expand_dims(last_sequence, axis=0)
    prediction = modelg.predict(last_sequence)
    print(prediction)
    predicted_price = scaler.inverse_transform(prediction)[0][0]
  
    predictions.append(round(float(predicted_price), 2))
    
# Print Predictions
if len(predictions) > 0:
    predictions_list = ['$'+str(d) for d in predictions]
    predictions_str = ', '.join(predictions_list)
    message = f'{STOCK} share price prediction(s) for next {len(predictions)} day(s) {predictions_str}'
    print(message)

GRU3df = pd.DataFrame({"Date":list(raw_price_df.index)[-PREDICTION_STEPS:],
                       "Actual Close":[round(x,2) for x in list(raw_price_df.Close.tail(PREDICTION_STEPS).values)],
                       "Predicted Close":predictions
                       })
GRU3df['Difference'] = [round(x,2) for x in np.array(LSTMdf['Actual Close']).astype(float)-predictions]
GRU3df["Model"] = 'GRU'
GRU3df["Optimizer"] = OPTIMIZER
GRU3df["Batch Size"] = BATCH_SIZE

GRU3df

In [ ]:
GRUav = round(np.mean(GRUdf.Difference),4)
GRU2av = round(np.mean(GRU2df.Difference),4)
GRU3av = round(np.mean(GRU3df.Difference),4)
print(f"Averages: GRU with Adam Optimizer: {GRUav}  -  GRU with RMSprop Optimizer: {GRU2av}  - GRU with SGD Optimizer: {GRU3av}")
GRUout = pd.concat([GRUdf,GRU2df,GRU3df]) 
GRUout

When using the GRU units, the SGD (Stochastic Gradient Descent) optimizer beats out the RMSProp and Adam optimizers. This is a bit different than the LSTM results. The same pattern of fewer observations leading to better model outcomes seems to hold here too. Here, too, we could spend much more time optimizing these models. Tuning hyperparameters, tuning optimzers, and batch sizes could well lead to much more prediction precision.

# Results and Visualization

Now that we have data from 2 models, and tree optimers for each, let's plot that data against actual closing price for the stock for the number of days.

In [ ]:
allout = pd.concat([LSTout,GRUout])
allout

In [ ]:
plt.plot(raw_price_df.index.tolist()[-PREDICTION_STEPS:],
         raw_price_df.Close.tail(PREDICTION_STEPS),
         lw=5,
         color='black')
plt.plot(LSTMdf.Date,LSTMdf['Predicted Close'])
plt.plot(LSTM2df.Date,LSTM2df['Predicted Close'])
plt.plot(LSTM3df.Date,LSTM3df['Predicted Close'])
plt.plot(GRUdf.Date,GRUdf['Predicted Close'])
plt.plot(GRU2df.Date,GRU2df['Predicted Close'])
plt.plot(GRU3df.Date,GRU3df['Predicted Close'])
plt.xlabel("Date")
plt.ylabel('Price')
plt.xticks(rotation="vertical")
plt.legend(['Actual','LSTM (Adam)','LSTM (RMSprop)','LSTM (SGD)','GRU (Adam)', 'GRU (RMSprop)', 'GRU (SGD)'], 
           fontsize='x-small')
plt.show()



# Conclusion

This notebook shows that a large bit of the variation in a stock's price can indeed be modeled by recurrent neural networks with LSTM or GRU units. Out of the box (untuned) the RMSprop optimizer seems to work well on these data with SGD coming ina close second. Important to notice that the predictions from these two optimizers don't typically agree on direction though. Some other interesting tidbits include the visual confirmation of the affect optimizers play in these models. The variation in the models isn't as uniform as the variation in the optimizers. For instance, the green and brown lines in the plot above mimic each other and are both SGD optimizer (one using LSTM model and one using GRU). The same can be gleaned from the orange and purple lines which are both RMSprop optimizer.

Caveat to results: there wasn't an earnings release or other news this week that would have adversely affected these models. Stock patterns are only a portion of the underlying stock pricing mechanisms. Please don't use this model to automate trading as the yfinance package is unsupported.

This notebook is in no way complete. If I had more time I'd choose to tune the hyperparameters of the models and optimizers. Doing so could lead to increased accuracy. Other things that deserve attention in future iterations include tuning the batch sizes, playing with the interval (1min, 1hour, 1day) market data sequences, and the models themselves (varying the number of layers and units).

If you like this notebook or learned something an upvote is appreciated.

In [ ]:
# Inspiration for this exploration came from this article:
# https://medium.com/@onersarpnalcin/predicting-the-stock-market-with-lstm-via-tensorflow-in-python-bullish-or-bearish-tomorrow-3c4f2fc03c51
    